In [1]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
df = (sqlContext.createDataFrame([("2012-06-13", 0.694),
                            ("2012-06-20", -2.669),
                            ("2012-06-27", 0.245)],
                             ["ts", "val"]).withColumn("ts", col("ts").cast("date").cast("timestamp")))

In [11]:
df.head(5)

[Row(ts=datetime.datetime(2012, 6, 13, 0, 0), val=0.694),
 Row(ts=datetime.datetime(2012, 6, 20, 0, 0), val=-2.669),
 Row(ts=datetime.datetime(2012, 6, 27, 0, 0), val=0.245)]

In [6]:
from pyspark.sql.functions import col, max as max_, min as min_

In [7]:
day = 60 * 60 * 24
epoch = (col("ts").cast("bigint") / day).cast("bigint") * day

with_epoch = df.withColumn("epoch", epoch)

min_epoch, max_epoch = with_epoch.select(min_("epoch"), max_("epoch")).first()

In [9]:
# Reference range 
ref = sqlContext.range(min_epoch, max_epoch + 1, day).toDF("epoch")

(ref.join(with_epoch, "epoch", "left")
    .orderBy("epoch")
    .withColumn("ts_resampled", col("epoch").cast("timestamp"))
    .show(15, False))

+----------+-------------------+------+-------------------+
|epoch     |ts                 |val   |ts_resampled       |
+----------+-------------------+------+-------------------+
|1339459200|2012-06-13 00:00:00|0.694 |2012-06-12 02:00:00|
|1339545600|null               |null  |2012-06-13 02:00:00|
|1339632000|null               |null  |2012-06-14 02:00:00|
|1339718400|null               |null  |2012-06-15 02:00:00|
|1339804800|null               |null  |2012-06-16 02:00:00|
|1339891200|null               |null  |2012-06-17 02:00:00|
|1339977600|null               |null  |2012-06-18 02:00:00|
|1340064000|2012-06-20 00:00:00|-2.669|2012-06-19 02:00:00|
|1340150400|null               |null  |2012-06-20 02:00:00|
|1340236800|null               |null  |2012-06-21 02:00:00|
|1340323200|null               |null  |2012-06-22 02:00:00|
|1340409600|null               |null  |2012-06-23 02:00:00|
|1340496000|null               |null  |2012-06-24 02:00:00|
|1340582400|null               |null  |2